
<a id='reshaping'></a>
{{ header }}

# Reshaping and pivot tables

## Reshaping by pivoting DataFrame objects

<img src="user_guide/../_static/reshaping_pivot.png" style="">

Data is often stored in so-called “stacked” or “record” format:

For the curious here is how the above `DataFrame` was created:

In [ ]:
import pandas.util.testing as tm

tm.N = 3


def unpivot(frame):
    N, K = frame.shape
    data = {'value': frame.to_numpy().ravel('F'),
            'variable': np.asarray(frame.columns).repeat(N),
            'date': np.tile(np.asarray(frame.index), K)}
    return pd.DataFrame(data, columns=['date', 'variable', 'value'])


df = unpivot(tm.makeTimeDataFrame())

To select out everything for variable `A` we could do:

But suppose we wish to do time series operations with the variables. A better
representation would be where the `columns` are the unique variables and an
`index` of dates identifies individual observations. To reshape the data into
this form, we use the `DataFrame.pivot()` method (also implemented as a
top level function `pivot()`):

If the `values` argument is omitted, and the input `DataFrame` has more than
one column of values which are not used as column or index inputs to `pivot`,
then the resulting “pivoted” `DataFrame` will have [hierarchical columns](advanced.ipynb#advanced-hierarchical) whose topmost level indicates the respective value
column:

You can then select subsets from the pivoted `DataFrame`:

Note that this returns a view on the underlying data in the case where the data
are homogeneously-typed.

>**Note**
>
>`pivot()` will error with a `ValueError: Index contains duplicate
entries, cannot reshape` if the index/column pair is not unique. In this
case, consider using `pivot_table()` which is a generalization
of pivot that can handle duplicate values for one index/column pair.


<a id='reshaping-stacking'></a>

## Reshaping by stacking and unstacking

<img src="user_guide/../_static/reshaping_stack.png" style="">

Closely related to the `pivot()` method are the related
`stack()` and `unstack()` methods available on
`Series` and `DataFrame`. These methods are designed to work together with
`MultiIndex` objects (see the section on [hierarchical indexing](advanced.ipynb#advanced-hierarchical)). Here are essentially what these methods do:

- `stack`: “pivot” a level of the (possibly hierarchical) column labels,
  returning a `DataFrame` with an index with a new inner-most level of row
  labels.  
- `unstack`: (inverse operation of `stack`) “pivot” a level of the
  (possibly hierarchical) row index to the column axis, producing a reshaped
  `DataFrame` with a new inner-most level of column labels.  


<img src="user_guide/../_static/reshaping_unstack.png" style="">

The clearest way to explain is by example. Let’s take a prior example data set
from the hierarchical indexing section:

The `stack` function “compresses” a level in the `DataFrame`’s columns to
produce either:

- A `Series`, in the case of a simple column Index.  
- A `DataFrame`, in the case of a `MultiIndex` in the columns.  


If the columns have a `MultiIndex`, you can choose which level to stack. The
stacked level becomes the new lowest level in a `MultiIndex` on the columns:

With a “stacked” `DataFrame` or `Series` (having a `MultiIndex` as the
`index`), the inverse operation of `stack` is `unstack`, which by default
unstacks the **last level**:


<a id='reshaping-unstack-by-name'></a>
<img src="user_guide/../_static/reshaping_unstack_1.png" style="">

If the indexes have names, you can use the level names instead of specifying
the level numbers:

<img src="user_guide/../_static/reshaping_unstack_0.png" style="">

Notice that the `stack` and `unstack` methods implicitly sort the index
levels involved. Hence a call to `stack` and then `unstack`, or vice versa,
will result in a **sorted** copy of the original `DataFrame` or `Series`:

The above code will raise a `TypeError` if the call to `sort_index` is
removed.


<a id='reshaping-stack-multiple'></a>

### Multiple levels

You may also stack or unstack more than one level at a time by passing a list
of levels, in which case the end result is as if each level in the list were
processed individually.

The list of levels can contain either level names or level numbers (but
not a mixture of the two).

### Missing data

These functions are intelligent about handling missing data and do not expect
each subgroup within the hierarchical index to have the same set of labels.
They also can handle the index being unsorted (but you can make it sorted by
calling `sort_index`, of course). Here is a more complex example:

As mentioned above, `stack` can be called with a `level` argument to select
which level in the columns to stack:

Unstacking can result in missing values if subgroups do not have the same
set of labels.  By default, missing values will be replaced with the default
fill value for that data type, `NaN` for float, `NaT` for datetimelike,
etc.  For integer types, by default data will converted to float and missing
values will be set to `NaN`.

New in version 0.18.0.

Alternatively, unstack takes an optional `fill_value` argument, for specifying
the value of missing data.

### With a MultiIndex

Unstacking when the columns are a `MultiIndex` is also careful about doing
the right thing:


<a id='reshaping-melt'></a>

## Reshaping by Melt

<img src="user_guide/../_static/reshaping_melt.png" style="">

The top-level `melt()` function and the corresponding `DataFrame.melt()`
are useful to massage a `DataFrame` into a format where one or more columns
are *identifier variables*, while all other columns, considered *measured
variables*, are “unpivoted” to the row axis, leaving just two non-identifier
columns, “variable” and “value”. The names of those columns can be customized
by supplying the `var_name` and `value_name` parameters.

For instance,

Another way to transform is to use the `wide_to_long()` panel data
convenience function. It is less flexible than `melt()`, but more
user-friendly.

## Combining with stats and GroupBy

It should be no shock that combining `pivot` / `stack` / `unstack` with
GroupBy and the basic Series and DataFrame statistical functions can produce
some very expressive and fast data manipulations.

## Pivot tables


<a id='reshaping-pivot'></a>
While `pivot()` provides general purpose pivoting with various
data types (strings, numerics, etc.), pandas also provides `pivot_table()`
for pivoting with aggregation of numeric data.

The function `pivot_table()` can be used to create spreadsheet-style
pivot tables. See the [cookbook](cookbook.ipynb#cookbook-pivot) for some advanced
strategies.

It takes a number of arguments:

- `data`: a DataFrame object.  
- `values`: a column or a list of columns to aggregate.  
- `index`: a column, Grouper, array which has the same length as data, or list of them.
  Keys to group by on the pivot table index. If an array is passed, it is being used as the same manner as column values.  
- `columns`: a column, Grouper, array which has the same length as data, or list of them.
  Keys to group by on the pivot table column. If an array is passed, it is being used as the same manner as column values.  
- `aggfunc`: function to use for aggregation, defaulting to `numpy.mean`.  


Consider a data set like this:

We can produce pivot tables from this data very easily:

The result object is a `DataFrame` having potentially hierarchical indexes on the
rows and columns. If the `values` column name is not given, the pivot table
will include all of the data that can be aggregated in an additional level of
hierarchy in the columns:

Also, you can use `Grouper` for `index` and `columns` keywords. For detail of `Grouper`, see [Grouping with a Grouper specification](groupby.ipynb#groupby-specify).

You can render a nice output of the table omitting the missing values by
calling `to_string` if you wish:


<dl style='margin: 20px 0;'>
<dt>Note that `pivot_table` is also available as an instance method on DataFrame,</dt>
<dd>
i.e. `DataFrame.pivot_table()`.

</dd>

</dl>


<a id='reshaping-pivot-margins'></a>

### Adding margins

If you pass `margins=True` to `pivot_table`, special `All` columns and
rows will be added with partial group aggregates across the categories on the
rows and columns:


<a id='reshaping-crosstabulations'></a>

## Cross tabulations

Use `crosstab()` to compute a cross-tabulation of two (or more)
factors. By default `crosstab` computes a frequency table of the factors
unless an array of values and an aggregation function are passed.

It takes a number of arguments

- `index`: array-like, values to group by in the rows.  
- `columns`: array-like, values to group by in the columns.  
- `values`: array-like, optional, array of values to aggregate according to
  the factors.  
- `aggfunc`: function, optional, If no values array is passed, computes a
  frequency table.  
- `rownames`: sequence, default `None`, must match number of row arrays passed.  
- `colnames`: sequence, default `None`, if passed, must match number of column
  arrays passed.  
- `margins`: boolean, default `False`, Add row/column margins (subtotals)  
- `normalize`: boolean, {‘all’, ‘index’, ‘columns’}, or {0,1}, default `False`.
  Normalize by dividing all values by the sum of values.  


Any `Series` passed will have their name attributes used unless row or column
names for the cross-tabulation are specified

For example:

If `crosstab` receives only two Series, it will provide a frequency table.

Any input passed containing `Categorical` data will have **all** of its
categories included in the cross-tabulation, even if the actual data does
not contain any instances of a particular category.

### Normalization

New in version 0.18.1.

Frequency tables can also be normalized to show percentages rather than counts
using the `normalize` argument:

`normalize` can also normalize values within each row or within each column:

`crosstab` can also be passed a third `Series` and an aggregation function
(`aggfunc`) that will be applied to the values of the third `Series` within
each group defined by the first two `Series`:

### Adding margins

Finally, one can also add margins or normalize this output.


<a id='reshaping-tile'></a>

<a id='reshaping-tile-cut'></a>

## Tiling

The `cut()` function computes groupings for the values of the input
array and is often used to transform continuous variables to discrete or
categorical variables:

If the `bins` keyword is an integer, then equal-width bins are formed.
Alternatively we can specify custom bin-edges:

New in version 0.20.0.

If the `bins` keyword is an `IntervalIndex`, then these will be
used to bin the passed data.:

In [ ]:
pd.cut([25, 20, 50], bins=c.categories)


<a id='reshaping-dummies'></a>

## Computing indicator / dummy variables

To convert a categorical variable into a “dummy” or “indicator” `DataFrame`,
for example a column in a `DataFrame` (a `Series`) which has `k` distinct
values, can derive a `DataFrame` containing `k` columns of 1s and 0s using
`get_dummies()`:

Sometimes it’s useful to prefix the column names, for example when merging the result
with the original `DataFrame`:

This function is often used along with discretization functions like `cut`:

See also `Series.str.get_dummies`.

`get_dummies()` also accepts a `DataFrame`. By default all categorical
variables (categorical in the statistical sense, those with object or
categorical dtype) are encoded as dummy variables.

All non-object columns are included untouched in the output. You can control
the columns that are encoded with the `columns` keyword.

Notice that the `B` column is still included in the output, it just hasn’t
been encoded. You can drop `B` before calling `get_dummies` if you don’t
want to include it in the output.

As with the `Series` version, you can pass values for the `prefix` and
`prefix_sep`. By default the column name is used as the prefix, and ‘_’ as
the prefix separator. You can specify `prefix` and `prefix_sep` in 3 ways:

- string: Use the same value for `prefix` or `prefix_sep` for each column
  to be encoded.  
- list: Must be the same length as the number of columns being encoded.  
- dict: Mapping column name to prefix.  


New in version 0.18.0.

Sometimes it will be useful to only keep k-1 levels of a categorical
variable to avoid collinearity when feeding the result to statistical models.
You can switch to this mode by turn on `drop_first`.

When a column contains only one level, it will be omitted in the result.

By default new columns will have `np.uint8` dtype.
To choose another dtype, use the `dtype` argument:

New in version 0.23.0.


<a id='reshaping-factorize'></a>

## Factorizing values

To encode 1-d values as an enumerated type use `factorize()`:

Note that `factorize` is similar to `numpy.unique`, but differs in its
handling of NaN:

>**Note**
>
>The following `numpy.unique` will fail under Python 3 with a `TypeError`
because of an ordering bug. See also
[here](https://github.com/numpy/numpy/issues/641).

```ipython
In [1]: x = pd.Series(['A', 'A', np.nan, 'B', 3.14, np.inf])
In [2]: pd.factorize(x, sort=True)
Out[2]:
(array([ 2,  2, -1,  3,  0,  1]),
 Index([3.14, inf, 'A', 'B'], dtype='object'))

In [3]: np.unique(x, return_inverse=True)[::-1]
Out[3]: (array([3, 3, 0, 4, 1, 2]), array([nan, 3.14, inf, 'A', 'B'], dtype=object))
```


>**Note**
>
>If you just want to handle one column as a categorical variable (like R’s factor),
you can use  `df["cat_col"] = pd.Categorical(df["col"])` or
`df["cat_col"] = df["col"].astype("category")`. For full docs on `Categorical`,
see the [Categorical introduction](categorical.ipynb#categorical) and the
API documentation.

## Examples

In this section, we will review frequently asked questions and examples. The
column names and relevant column values are named to correspond with how this
DataFrame will be pivoted in the answers below.

### Pivoting with single aggregations

Suppose we wanted to pivot `df` such that the `col` values are columns,
`row` values are the index, and the mean of `val0` are the values? In
particular, the resulting DataFrame should look like:

>**Note**
>
>col   col0   col1   col2   col3  col4
row
row0  0.77  0.605    NaN  0.860  0.65
row2  0.13    NaN  0.395  0.500  0.25
row3   NaN  0.310    NaN  0.545   NaN
row4   NaN  0.100  0.395  0.760  0.24

This solution uses `pivot_table()`. Also note that
`aggfunc='mean'` is the default. It is included here to be explicit.

Note that we can also replace the missing values by using the `fill_value`
parameter.

Also note that we can pass in other aggregation functions as well. For example,
we can also pass in `sum`.

Another aggregation we can do is calculate the frequency in which the columns
and rows occur together a.k.a. “cross tabulation”. To do this, we can pass
`size` to the `aggfunc` parameter.

### Pivoting with multiple aggregations

We can also perform multiple aggregations. For example, to perform both a
`sum` and `mean`, we can pass in a list to the `aggfunc` argument.

Note to aggregate over multiple value columns, we can pass in a list to the
`values` parameter.

Note to subdivide over multiple columns we can pass in a list to the
`columns` parameter.


<a id='reshaping-explode'></a>

## Exploding a list-like column

New in version 0.25.0.

Sometimes the values in a column are list-like.

We can ‘explode’ the `values` column, transforming each list-like to a separate row, by using `explode()`. This will replicate the index values from the original row:

You can also explode the column in the `DataFrame`.

`Series.explode()` will replace empty lists with `np.nan` and preserve scalar entries. The dtype of the resulting `Series` is always `object`.

Here is a typical usecase. You have comma separated strings in a column and want to expand this.

Creating a long form DataFrame is now straightforward using explode and chained operations